In [1]:
import pandas as pd
import numpy as np

In [2]:
# --- Cargar los datos necesarios ---
FEATURED_DATA_PATH = '../data/processed/featured_player_dataset.csv'
df = pd.read_csv(FEATURED_DATA_PATH)

RAW_DATA_DIR = '../data/raw/davidcariboo_dataset/'
valuations_df = pd.read_csv(f'{RAW_DATA_DIR}player_valuations.csv')
competitions_df = pd.read_csv(f'{RAW_DATA_DIR}competitions.csv')

print("Datos cargados. Dimensiones del dataset principal:", df.shape)

Datos cargados. Dimensiones del dataset principal: (6280, 25)


In [3]:
# --- Diagnóstico: Encontrar el nombre de columna correcto ---
# Vamos a verificar el nombre exacto de la columna de la competición en el archivo de valoraciones.
print("\n--- DIAGNÓSTICO ---")
print("Columnas disponibles en 'valuations_df':", valuations_df.columns.tolist())
# El nombre que buscamos es 'player_club_domestic_competition_id'. Si es diferente, lo veremos aquí.


--- DIAGNÓSTICO ---
Columnas disponibles en 'valuations_df': ['player_id', 'date', 'market_value_in_eur', 'current_club_id', 'player_club_domestic_competition_id']


In [4]:
# =============================================================================
# Feature 1: Fuerza de la Liga (League Strength Factor)
# =============================================================================

# --- Paso 1: Crear el mapa de fuerza de las ligas ---
league_strength_map = {
    'premier-league':   1.00,
    'laliga':           0.90,
    'serie-a':          0.88,
    'bundesliga':       0.87,
    'ligue-1':          0.80,
    'liga-nos':         0.78,
    'eredivisie':       0.76,
    'super-lig':        0.68,
    'jupiler-pro-league': 0.65,
    'premier-liga':     0.50
}

# --- Paso 2: Obtener la información más reciente de valoración (incluida la liga) ---
valuations_df['date'] = pd.to_datetime(valuations_df['date'])
latest_valuations = valuations_df.sort_values(by='date', ascending=False).drop_duplicates(subset='player_id')

# --- ¡¡¡CORRECCIÓN CLAVE!!! ---
# Definimos el nombre de la columna que vamos a añadir.
COMPETITION_ID_COL = 'player_club_domestic_competition_id'

# --- Paso 3: Unir la información de la liga a nuestro dataframe principal ---
# Solo unimos las columnas que necesitamos: el ID del jugador y el ID de su competición.
df = pd.merge(
    df,
    latest_valuations[['player_id', COMPETITION_ID_COL]],
    on='player_id',
    how='left'
)

# --- Paso 4: Mapear el ID de la competición al nombre y luego al factor de fuerza ---
competition_id_to_name_map = competitions_df.set_index('competition_id')['name'].to_dict()
# Usamos la variable COMPETITION_ID_COL para asegurar que no hay errores de tipeo.
df['competition_name'] = df[COMPETITION_ID_COL].map(competition_id_to_name_map)
df['league_strength_factor'] = df['competition_name'].map(league_strength_map).fillna(0.4)

print("\nFeature 'league_strength_factor' creada exitosamente.")

# --- Paso 5: Validar el resultado ---
print("\nValidación del factor de fuerza de liga:")
validation_df = df[df['competition_name'].isin(['premier-league', 'serie-a', 'super-lig'])]
print(validation_df[['name', 'competition_name', 'league_strength_factor']].sample(5, random_state=1))


Feature 'league_strength_factor' creada exitosamente.

Validación del factor de fuerza de liga:
                     name competition_name  league_strength_factor
2821  Gaetano Castrovilli          serie-a                    0.88
5366           James Hill   premier-league                    1.00
4789         Roger Ibañez          serie-a                    0.88
5678             Chermiti   premier-league                    1.00
713   Stephan El Shaarawy          serie-a                    0.88


In [5]:
# =============================================================================
# Feature 2: Prestigio del Club (Club Tier)
# =============================================================================
tier1_clubs = [
    'Real Madrid', 'FC Barcelona', 'Atlético de Madrid', 'Manchester City', 'Liverpool FC',
    'Arsenal FC', 'Manchester United', 'Chelsea FC', 'Bayern Munich', 'Borussia Dortmund',
    'Bayer 04 Leverkusen', 'Juventus FC', 'Inter Milan', 'AC Milan', 'Paris Saint-Germain',
    'AFC Ajax', 'FC Porto', 'SL Benfica'
]
df['club_tier'] = df['current_club_name'].apply(lambda name: 1 if name in tier1_clubs else 0)
print("\nFeature 'club_tier' creada exitosamente.")

# --- Validación ---
print("\nValidación del Tier de Clubes:")
validation_clubs = ['Manchester City', 'Real Madrid', 'Burnley FC', 'Getafe CF', 'AC Milan']
club_tier_validation = df[df['current_club_name'].isin(validation_clubs)]
print(club_tier_validation[['name', 'current_club_name', 'club_tier']].drop_duplicates('current_club_name'))


Feature 'club_tier' creada exitosamente.

Validación del Tier de Clubes:
                name current_club_name  club_tier
1037  Nathan Redmond        Burnley FC          0


In [6]:
# --- Limpieza Final y Guardado ---
# Eliminamos columnas intermedias que ya no necesitamos para el modelado
cols_to_drop = ['player_club_domestic_competition_id', 'competition_name']
df = df.drop(columns=cols_to_drop)

OUTPUT_PATH = '../data/processed/contextual_featured_dataset.csv'
df.to_csv(OUTPUT_PATH, index=False)
print(f"\nDataset con features contextuales guardado en: {OUTPUT_PATH}")
print(f"Dimensiones finales del dataset: {df.shape}")


Dataset con features contextuales guardado en: ../data/processed/contextual_featured_dataset.csv
Dimensiones finales del dataset: (6280, 27)
